In [ ]:
struct Trajectory
    nrows::Int64
    nframes::Int64
    dfs::Vector{Matrix{Float64}}
    time::Vector{Float64}
end

function Base.getindex(traj::Trajectory, i::Int64)
    vcat(traj.dfs[i][end:end, :], traj.dfs[i][1:end-1, :])
end

function Base.getindex(traj::Trajectory, frames::Vector{Int64})
    Trajectory(traj.nrows, length(frames), [traj.dfs[i] for i in frames])
end
function Base.getindex(traj::Trajectory, frames::Union{Vector{Int64}, UnitRange, StepRange})
    if isa(frames, Union{UnitRange, StepRange})
        frames = collect(frames)
    end
    traj[frames]
end

function Base.lastindex(traj::Trajectory)
    traj.nframes
end

function Base.length(traj::Trajectory)
    length(traj.dfs)
end

Base.iterate(traj::Trajectory, state=1) = state > length(traj) ? nothing : (traj[state], state+1)

function Base.show(io::IO, traj::Trajectory)
    println(io, "Number of frames: $(traj.nframes)")
    println(io, "Size of frame: $(traj.nrows)")
    println(io, "Dataframes:")
    for (ind, df) in enumerate(traj.dfs)
        if (ind <= 3) | (ind > (traj.nframes - 3))
            println("frame $ind: $(size(df)), time: $(traj.time[ind])")
        elseif ind < 5
            println("...")
        else
        end
    end
end

function Trajectory(filename::String)
    out = open(filename) do f
        x = readline(f)
        N = parse(Int64, x)
        x = readline(f)
        time :: Vector{Float64} = [] 
        get_t = (x) -> parse(Float64, split(split(x, "Time=")[2])[1])
        push!(time, get_t(x))
        function makevector(ind)
            reshape(map(x->parse(Float64, x), split(readline(f))), 1, :)
        end
        function check()
            if eof(f)
                return true, nothing
            end
            return false, readline(f)
        end
        df :: Vector{Matrix{Float64}} = [] 
        while !eof(f)
            push!(df, mapreduce(makevector, vcat, 1:N))
            bool, x = check()
            if bool
                break
            end
            bool, x = check()
            push!(time, get_t(x))
            if bool
                break
            end
        end
        (N, length(df), df, time)
    end
    Trajectory(out...)
end


In [ ]:
traj = Trajectory("chain.data")

In [ ]:
# function kinetic_energy(traj::Trajectory, frames::Vector{Int64}) 
#     kinetic_energy(traj[frames])
# end

# function kinetic_energy(traj::Trajectory, frames::Union{UnitRange, StepRange, Nothing}) 
#     if isa(frames, Nothing)
#         frames = 1:traj.nframes
#     end
#     frames = collect(frames)
#     kinetic_energy(traj, frames)
# end

# function kinetic_energy(traj::Trajectory)
#     map((x) -> +(x...), kinetic_energy_(traj))
# end

# function kinetic_energy_(traj::Trajectory)
#     m = 1.0
#     l = 1.0
#     I = 1/12 * m * l^2 
#     lin_ke(v) = 0.5 * m * sum(v.^2)
#     ang_ke(w) = 0.5 * I * sum(w.^2)
#     get_w(r, v) = r[1]*v[2] - r[2]*v[1]
#     map((df) -> begin
#             v = [df[i+1, 5:6] - df[i, 5:6] for i in 2:traj.nrows-1]
#             r = [df[i+1, 3:4] - df[i, 3:4] for i in 2:traj.nrows-1]
#             v_ = [(df[i, 5:6] + df[i, 5:6]) / 2 for i in 2:traj.nrows-1]
#             l_ke = sum(lin_ke.(v_))
#             w = get_w.(r, v)
#             a_ke = sum(ang_ke.(w))
#             l_ke, a_ke
#             end, 
#             traj)
# end

# function potential_energy(traj::Trajectory, frames::Vector{Int64}) 
#     potential_energy(traj[frames])
# end

# function potential_energy(traj::Trajectory, frames::Union{UnitRange, StepRange, Nothing}) 
#     if isa(frames, Nothing)
#         frames = 1:traj.nframes
#     end
#     frames = collect(frames)
#     potential_energy(traj, frames)
# end

# function potential_energy(traj::Trajectory)
#     m = 1.0
#     g = 10.0
#     map((df) -> begin
#             h = (df[2:end-1, 4] + df[3:end, 4])/2
#             pe = m*g*sum(h)
#             end, 
#             traj)
# end

# function plot_energy(traj::Trajectory; frames=nothing)
#     plot_energy(traj, frames)
# end

# function plot_energy(traj::Trajectory, frames)
#     PE = potential_energy(traj, frames)
#     KE = hcat(map(x -> [x...], kinetic_energy_(traj))...)'
    
#     if isa(frames, Nothing)
#         n = traj.nframes
#     else
#         n = length(frames)
#     end
    
#     PE = PE .- PE[1]
#     KE = KE .- KE[1:1, :]
#     TE = PE .+ sum(KE, dims=2)
    
#     KE_l = KE[:, 1]
#     KE_a = KE[:, 2]
    
#     fig = plot(1:n, PE, label="PE")
#     plot!(1:n, KE_l, label="KE_l")
#     plot!(1:n, KE_a, label="KE_a")
#     plot!(1:n, KE_a .+ KE_l, label="KE")
#     plot!(1:n, TE, label="TE")

#     display(fig)
# end

# using Plots

# traj = Trajectory("chain.data")
# plot_energy(traj)


In [ ]:
# filename1 = "../hpc/results/2-4-hybrid-lgnn/0/actual_0_0.ovito"
# filename2 = "../hpc/results/2-4-hybrid-lgnn/0/pred_0_0.ovito"

filename = "chain_exp/chain_T1"
outfilename = filename*".mp4"
mkpath(dirname(outfilename))
traj = Trajectory(filename*"_free.data")
traj_model = Trajectory(filename*"_model_free.data")

using GLMakie
 
function clearaxis!(ax)
    hidespines!(ax)
    hidedecorations!(ax)

end

time = Observable(1)

N = traj.nrows

mask = collect(1:N)
# mask[end-2] = 3
# mask[end-1] = 1
# mask[end] = N-1


xs = @lift(traj[$time][mask, 3])
ys = @lift(traj[$time][mask, 4])

xs_model = @lift(traj_model[$time][mask, 3])
ys_model = @lift(traj_model[$time][mask, 4])


# XS = lift((t)->draw_zigzag!(to_value(xsl2_), to_value(ysl2_), out=1, width=0.05), time)
# YS = lift((t)->draw_zigzag!(to_value(xsl2_), to_value(ysl2_), out=2, width=0.05), time)

# XS_ = lift((t)->draw_zigzag!(to_value(xsl1_), to_value(ysl1_), out=1, width=0.1), time)
# YS_ = lift((t)->draw_zigzag!(to_value(xsl1_), to_value(ysl1_), out=2, width=0.1), time)

# fig = Figure(resolution = (500, 500))
# ax = Axis(fig)

get_time = (t) -> "time = $(round(traj.time[t]; sigdigits=8)) \n dt = $(round(traj.time[t] - traj.time[max(1, t - 1)]; sigdigits=8))"

# fig = Figure(resolution = (800, 800))
# ax = Axis(fig[1, 1], aspect = 1)

fig, ax = lines(xs, ys, linewidth=6, color=:lightblue,
    axis = (title = @lift(get_time($time)), aspect=1), figure=(resolution=(5000, 5000),))

n = 10

index = hcat([n+2, n+3, 2*(n+1)+1],
[2*(n+1)+2, 3*(n+1)+1, 3*(n+1)+2])


p1_x = @lift(traj[$time][index[1, :], 3])
p1_y = @lift(traj[$time][index[1, :], 4])

p2_x = @lift(traj[$time][index[2, :], 3])
p2_y = @lift(traj[$time][index[2, :], 4])

p3_x = @lift(traj[$time][index[3, :], 3])
p3_y = @lift(traj[$time][index[3, :], 4])

lines!(p1_x, p1_y, linewidth=6, color=:lightblue)
lines!(p2_x, p2_y, linewidth=6, color=:lightblue)
lines!(p3_x, p3_y, linewidth=6, color=:lightblue)
    
scatter!([0], [0], color = :brown, markersize=10)
scatter!(xs, ys, color = :red, markersize=10, label="Ground truth")


lines!(xs_model, ys_model, linewidth=1, color=:black, label="Prediction")

p1_x = @lift(traj_model[$time][index[1, :], 3])
p1_y = @lift(traj_model[$time][index[1, :], 4])

p2_x = @lift(traj_model[$time][index[2, :], 3])
p2_y = @lift(traj_model[$time][index[2, :], 4])

p3_x = @lift(traj_model[$time][index[3, :], 3])
p3_y = @lift(traj_model[$time][index[3, :], 4])

lines!(p1_x, p1_y, linewidth=1, color=:black)
lines!(p2_x, p2_y, linewidth=1, color=:black)
lines!(p3_x, p3_y, linewidth=1, color=:black)

scatter!(xs_model, ys_model, color = :white, markersize=5, label="Prediction")


# # lines!(xsl1_, ysl1_, color = :magenta, linewidth = 10)
# lines!(XS_, YS_, color = :brown, linewidth = 1)

# if pred
#     lines!(xsl2, ysl2, color = :black, linewidth = 4, zorder=5)
#     # lines!(xsl2_, ysl2_, color = :black, linewidth = 2, zorder=5)
#     lines!(XS, YS, color = :black, linewidth = 1, zorder=5)
# end

# scatter!(xss12, yss12, color = :brown, markersize = 10*3/N)
# scatter!(xss11, yss11, color = :red, markersize = 50*3/N, label="Ground truth")

# if pred
#     scatter!(xss21, yss21, color = :black, markersize = 20*3/N, label="Prediction")
# end

STEPS = length(traj)

set_theme!()
clearaxis!(ax)


width = 4
shift = 3 #2 / sqrt(2)

xlims!(ax, -width+shift , width+shift)
ylims!(ax, -width, width)

axislegend(framevisible = true, framewidth=0, orientation=:horizontal, bgcolor="#DDDDDD", position=:ct, merge=true)


framerate = 60
timestamps = 1:STEPS


record(fig, outfilename, timestamps;
        framerate = framerate) do t
    clearaxis!(ax)
    time[] = t
end

fig

In [ ]:
# filename1 = "../hpc/results/2-4-hybrid-lgnn/0/actual_0_0.ovito"
# filename2 = "../hpc/results/2-4-hybrid-lgnn/0/pred_0_0.ovito"

outfilename = "temp_animation.mp4"
mkpath(dirname(outfilename))
traj = Trajectory("spring_rod_data_n3.data")

using GLMakie
 
function clearaxis!(ax)
    hidespines!(ax)
    hidedecorations!(ax)
end

time = Observable(1)

k = 10

xs = @lift(traj[$time][1:end, 3]*k)
ys = @lift(traj[$time][1:end, 4]*k)

# XS = lift((t)->draw_zigzag!(to_value(xsl2_), to_value(ysl2_), out=1, width=0.05), time)
# YS = lift((t)->draw_zigzag!(to_value(xsl2_), to_value(ysl2_), out=2, width=0.05), time)

# XS_ = lift((t)->draw_zigzag!(to_value(xsl1_), to_value(ysl1_), out=1, width=0.1), time)
# YS_ = lift((t)->draw_zigzag!(to_value(xsl1_), to_value(ysl1_), out=2, width=0.1), time)

# fig = Figure(resolution = (500, 500))
# ax = Axis(fig)

get_time = (t) -> "time = $(round(traj.time[t]; sigdigits=8)) \n dt = $(round(traj.time[t] - traj.time[max(1, t - 1)]; sigdigits=8))"

fig, ax = lines(xs, ys, linewidth=10, color=1:traj.nrows, colormap=:darktest,
    axis = (title = @lift(get_time($time)), ))
scatter!([0], [0], color = :brown, markersize=4)
scatter!(xs, ys, color = :red, markersize=14)



# # lines!(xsl1_, ysl1_, color = :magenta, linewidth = 10)
# lines!(XS_, YS_, color = :brown, linewidth = 1)

# if pred
#     lines!(xsl2, ysl2, color = :black, linewidth = 4, zorder=5)
#     # lines!(xsl2_, ysl2_, color = :black, linewidth = 2, zorder=5)
#     lines!(XS, YS, color = :black, linewidth = 1, zorder=5)
# end

# scatter!(xss12, yss12, color = :brown, markersize = 10*3/N)
# scatter!(xss11, yss11, color = :red, markersize = 50*3/N, label="Ground truth")

# if pred
#     scatter!(xss21, yss21, color = :black, markersize = 20*3/N, label="Prediction")
# end

STEPS = length(traj)

set_theme!()
clearaxis!(ax)
width = 100
xlims!(ax, -0.5*width , 0.5*width)
ylims!(ax, -0.5*width, 0.5*width)
# axislegend(framevisible = false)
# Legend(fig, ax, framevisible = false, position = :rt)

framerate = 60
timestamps = 1:STEPS


record(fig, outfilename, timestamps;
        framerate = framerate) do t
    clearaxis!(ax)
    time[] = t
end


fig

In [ ]:
function draw_zigzag!(xob, yob; out=3, num=10, width=0.2)
    x = to_value(xob)
    y = to_value(yob)
    
    out1 = []
    out2 = []
    for i in 1:length(x)-1
        rx = x[i+1] - x[i]
        ry = y[i+1] - y[i]
        l = sqrt(rx^2+ry^2)
        tx = rx/l
        ty = ry/l
        p0x = x[i] + rx*0.1
        p1x = x[i] + rx*0.9
        p0y = y[i] + ry*0.1
        p1y = y[i] + ry*0.9
        
        x1 = collect(range(p0x, p1x, length=2*num))
        y1 = collect(range(p0y, p1y, length=2*num))

        y1[2:2:length(y1)-1] .+= width * tx 
        y1[3:2:length(y1)-1] .-= width * tx 
        x1[2:2:length(x1)-1] .-= width * ty 
        x1[3:2:length(x1)-1] .+= width * ty 

        out1 = [out1 ..., x[i], x1..., x[i+1]] 
        out2 = [out2..., y[i], y1..., y[i+1]]
    end
    if out==1
        return out1
    elseif out==2
        return out2
    end
    out1, out2
end
